# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step2 加载数据

In [4]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [6]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [7]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [9]:
for i in range(10):
    print(trainset[i])

('环境地理位置很好，交通方便。房间舒适，服务很好', 1)
('房间还不错，床铺比较干净，这是优点。缺点：服务较差，有时候叫服务员服务员居然说你为什么不能自己动手啊，我在国内其他任何酒店从未听过。携程定的房只限二层，来之前没有人通知过。', 0)
('其它都可以但本次提供的房间不好窗户离墙只有几公分，应事先和客人打个招呼说明一下，否则很郁闷。', 1)
('处于无锡商业街附近，离火车站很近。非常方便。附近的美食店也可步行到达。很方便。房间中规中矩，纯木色装修，很舒服。绿色地毯个人不喜欢，不过也是很多星级酒店的首选。通过携程预定，可以免费上网（50元/天）比较惊喜。另有二个饮料赠送。开心。洗浴小包装味道很好。很喜欢。拖鞋也很漂亮。服务好，行动快。最有意思的是，进门的灯是找不到开关的，打电话询问，回答说，那个灯是应急灯，是关不了的。只能用总电源开关关掉，如果您一定要关闭的话，我们可以让工程部去帮您把灯泡卸下来。可爱死了。笑坏我了。大堂比较小。不过电梯很快。早餐品种还可以，中日西式，日式的味噌汤竟然还可以自己加料，很喜欢。不知不觉写了这么多，总之一句评价：中规中矩，五星标准。位置便利，商务可选。宾馆反馈2008年8月8日：感谢您对本酒店的支持！最近酒店有新的客房周末促销推出，我们将一如既往地保持5星级的服务标准，欢迎您再次下榻，衷心祝您身体健康，工作顺利!', 1)
('订的市景双床房，免费升级到豪华双床房，感觉很好。服务台工作人员耐心有礼貌，态度很好。房间情况：优点：卫生间较宽敞，干净，力士的洗浴用品，环保好用。房间设施用品齐全，熨衣板、雨伞、文具都为客人想到了。空调冷气足，而且出风口比较干净，没有特殊的气味。床柔软，两个长枕一个方枕，看电视靠着很舒服。电视频道清晰。checkin和checkout速度较快。缺点：房间设施比较陈旧，需要进行重新装修。房间有点小，像三星级规模。这里的服务确实很好，工作人员态度都很温和，面带微笑，将硬件上的稍许不足都弥补上了。以后还会考虑入住。宾馆反馈2008年8月5日：非常感谢您对我们酒店的点评。您对我们的肯定和支持，是我们工作的动力。酒店会不断的提升酒店的品质，提高宾客居住的舒适度，为宾客营造轻松、愉悦的旅居环境。酒店另有按高星级标准全新装修的商务豪华房，宽敞精美。建议您下次入住时，可以选择该房型。感谢您的宝贵意见，我们衷心期待您再次

## Step5 创建Dataloader

In [30]:
import torch

tokenizer = AutoTokenizer.from_pretrained("rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [31]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [32]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 2769,  812,  ...,    0,    0,    0],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 6392, 3177,  ...,    0,    0,    0],
        ...,
        [ 101, 3302, 1218,  ...,    0,    0,    0],
        [ 101, 2600,  860,  ...,  752, 2141,  102],
        [ 101, 1765, 4415,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1

## Step6 创建模型及优化器

In [39]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3/")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of the model checkpoint at ./rbt3/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3/ a

In [40]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [41]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [42]:
train()

ep: 0, global_step: 0, loss: 0.8356314301490784
ep: 0, global_step: 100, loss: 0.1966220587491989
ep: 0, global_step: 200, loss: 0.33055591583251953
ep: 0, acc: 0.8801546096801758
ep: 1, global_step: 300, loss: 0.31511273980140686
ep: 1, global_step: 400, loss: 0.2748050391674042
ep: 1, acc: 0.9046391248703003
ep: 2, global_step: 500, loss: 0.13369111716747284
ep: 2, global_step: 600, loss: 0.16734185814857483
ep: 2, acc: 0.894329845905304


## Step9 模型预测

In [45]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [48]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [49]:
pipe(sen)

[{'label': '好评！', 'score': 0.9974019527435303}]